# Example use of Bayesian Regression using MCMC and SVI

In [1]:
# Create 1D data for fit
import numpy as np
import plotly.graph_objects as go

# 1D Function to predict
def f(x):
    return x + x

#  First the noiseless case
X = np.atleast_2d(np.random.uniform(0, 10.0, size=100)).T
X = X.astype(np.float32)

# Observations
y = f(X).ravel()

dy = 1.5 + 1.0 * np.random.random(y.shape)
noise = np.random.normal(0, dy)
y += noise
y = y.reshape(-1,1).astype(np.float32)

# Create random data with numpy
import numpy as np
np.random.seed(1)


fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=X.flatten(), y=y.flatten(),
                    mode='markers',
                    name='Function to Predict'))
fig.update_layout(title='1D Function to Predict',
                  height=600,
                  width=800,
                  template="plotly_white")

fig.show()

In [25]:
# Define model prior

from pyro.infer.autoguide import AutoDiagonalNormal, AutoNormal
from pyro.infer import Predictive, SVI, Trace_ELBO
import pyro
import torch
from torch.distributions import constraints
import pyro.distributions as dist
import pyro.optim as optim
import sys
import time
from bayesian_regression_mcmc_svi import bay_reg_mcmc, bay_reg_svi

def prior_model(T, y_obs=None):

    # priors
    w = pyro.sample('w', pyro.distributions.Normal(loc=1, scale=10))
    b = pyro.sample('b', pyro.distributions.Normal(loc=1, scale=10))
    
    mean = w*T + b
    
    sigma=pyro.sample("sigma",dist.Uniform(0.,10.))
      
    with pyro.plate("data", len(T)):
        obs=pyro.sample("obs", dist.Normal(mean, sigma), obs=y_obs)
    return mean

# Initiate models, fit models, predict

results_dict = {}

#MCMC
pyro.clear_param_store()
start=time.time()
bay_reg_mc=bay_reg_mcmc(prior_model, num_samples=500, warmup_steps=100)
bay_reg_mc.fit(X,y, verbose=True)
    
X_test=np.linspace(0,10,11)
y_conv,y_std_conv,y_pred,y_std_pred=bay_reg_mc.predict(X_test)
end=round(time.time()-start,2)
results_dict['MCMC']={'X_pred':X_test,'y_conv':y_conv,'y_std_conv':y_std_conv,'y_pred':y_pred,'y_std_pred':y_std_pred,'elapsed time':end}

# SVI Model

# Define SVI guide if desired
# If SVI guide is not defined a guide autonormal guide is computed
def custom_guide(T,test_y=None):
    w_loc = pyro.param('w_loc', torch.tensor(1.))
    w_scale = pyro.param('w_scale', torch.tensor(10.),
                         constraint=constraints.positive)
    b_loc = pyro.param('b_loc', torch.tensor(1.))
    b_scale = pyro.param('b_scale', torch.tensor(10.),
                         constraint=constraints.positive)
    
    sigma_loc = pyro.param('sigma_loc', torch.tensor(0.),
                         constraint=constraints.positive)
    sigma_scale = pyro.param('sigma_scale', torch.tensor(1.),
                         constraint=constraints.positive)
    
    w = pyro.sample('w',dist.Normal(w_loc, w_scale))
    b = pyro.sample('b',dist.Normal(b_loc, b_scale))
    sigma=pyro.sample("sigma",dist.Normal(sigma_loc, sigma_scale))
    f_x = w*T + b
    
    return f_x

pyro.clear_param_store()
start=time.time()


bay_reg_vi=bay_reg_svi(prior_model, guide=None, epochs=1000)
bay_reg_vi.fit(X,y, verbose=True)
    
X_test=np.linspace(0,10,11)
y_conv,y_std_conv,y_pred,y_std_pred=bay_reg_vi.predict(X_test)
end=round(time.time()-start,2)

results_dict['SVI']={'X_pred':X_test,'y_conv':y_conv,'y_std_conv':y_std_conv,'y_pred':y_pred,'y_std_pred':y_std_pred,'elapsed time':end}

Sample: 100%|██████████| 600/600 [00:32, 18.54it/s, step size=5.79e-02, acc. prob=0.964]
Elbo loss: nan
Elbo loss: nan
Elbo loss: inf
Elbo loss: 29646.55834853649
Elbo loss: nan
Elbo loss: nan
Elbo loss: nan
Elbo loss: nan
Elbo loss: 94871.08241093159
Elbo loss: 35241.14662563801


In [26]:
# Display Results

import matplotlib.pyplot as plt

fig = go.Figure()

# Add Function to Predict
fig.add_trace(go.Scatter(x=X.flatten(), y=y.flatten(),
                    mode='markers',
                    name='Function to Predict', marker_color='black')), 

# Add predictions of fitted models

colors=plt.cm.Set1(np.linspace(0, 1, len(results_dict)))
for i, model in enumerate(results_dict):


    color=f"rgba({colors[i][0]},{colors[i][1]},{colors[i][2]},{colors[i][3]})"
    color_fill_conv=f"rgba({colors[i][0]},{colors[i][1]},{colors[i][2]},{0.5})"
    color_fill_pred=f"rgba({colors[i][0]},{colors[i][1]},{colors[i][2]},{0.3})"

    X_pred=results_dict[model]['X_pred']
    y_conv=results_dict[model]['y_conv']
    y_conv_lower=results_dict[model]['y_conv']-results_dict[model]['y_std_conv'][:,0].reshape(-1,1)
    y_conv_upper=results_dict[model]['y_conv']+results_dict[model]['y_std_conv'][:,1].reshape(-1,1)

    y_pred=results_dict[model]['y_pred']
    y_pred_lower=results_dict[model]['y_pred']-results_dict[model]['y_std_pred'][:,0].reshape(-1,1)
    y_pred_upper=results_dict[model]['y_pred']+results_dict[model]['y_std_pred'][:,1].reshape(-1,1)


    elapsed_time=results_dict[model]['elapsed time']

    # median
    fig.add_trace(go.Scatter(x=X_pred.flatten(), y=y_conv.flatten(),
                    mode='lines',
                    name=f'{model} median {elapsed_time}', marker_color=color))
    # convidence intervals

    fig.add_trace(go.Scatter(x=X_pred,y=y_conv_upper,mode='lines',name=f'{model} upper conv interval',fill=None, fillcolor=color_fill_conv, line_color=color_fill_conv))
    fig.add_trace(go.Scatter(x=X_pred,y=y_conv_lower,mode='lines',name=f'{model} lower conv interval',fill='tonexty',fillcolor=color_fill_conv, line_color=color_fill_conv))
    # prediction intervals

    fig.add_trace(go.Scatter(x=X_pred,y=y_pred_upper.flatten(),mode='lines',name=f'{model} upper pred interval',fill=None, fillcolor=color_fill_pred, line_color=color_fill_pred))
    fig.add_trace(go.Scatter(x=X_pred,y=y_pred_lower.flatten(),mode='lines',name=f'{model} lower pred interval',fill='tonexty',fillcolor=color_fill_pred, line_color=color_fill_pred))



fig.update_layout(title='Comparison of model predictions',
                  height=600,
                  width=800,
                  template="plotly_white")
fig.show()